In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  # noqa

In [9]:
dataset = 'Cora'
path = path = '../data/geometric/CORA'
dataset = Planetoid(path, dataset, T.NormalizeFeatures())
data = dataset[0]

In [19]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, improved=False)
        self.conv2 = GCNConv(16, dataset.num_classes, improved=False)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [20]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [21]:
best_val_acc = test_acc = 0
for epoch in range(1, 100):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))

Epoch: 001, Train: 0.1571, Val: 0.0740, Test: 0.0940
Epoch: 002, Train: 0.3214, Val: 0.1260, Test: 0.1550
Epoch: 003, Train: 0.6857, Val: 0.4500, Test: 0.4820
Epoch: 004, Train: 0.6643, Val: 0.5200, Test: 0.5520
Epoch: 005, Train: 0.6714, Val: 0.5640, Test: 0.5820
Epoch: 006, Train: 0.7500, Val: 0.5900, Test: 0.6230
Epoch: 007, Train: 0.7929, Val: 0.6220, Test: 0.6370
Epoch: 008, Train: 0.8429, Val: 0.6220, Test: 0.6370
Epoch: 009, Train: 0.8429, Val: 0.6220, Test: 0.6370
Epoch: 010, Train: 0.8429, Val: 0.6220, Test: 0.6370
Epoch: 011, Train: 0.8357, Val: 0.6220, Test: 0.6370
Epoch: 012, Train: 0.8429, Val: 0.6220, Test: 0.6370
Epoch: 013, Train: 0.8500, Val: 0.6220, Test: 0.6370
Epoch: 014, Train: 0.8571, Val: 0.6220, Test: 0.6370
Epoch: 015, Train: 0.8929, Val: 0.6220, Test: 0.6370
Epoch: 016, Train: 0.9071, Val: 0.6260, Test: 0.6470
Epoch: 017, Train: 0.9214, Val: 0.6480, Test: 0.6640
Epoch: 018, Train: 0.9357, Val: 0.6620, Test: 0.6820
Epoch: 019, Train: 0.9286, Val: 0.6720, Test: 